# Build TIPS Timeseries YTM

### CRSP Treasury Manual

https://www.crsp.org/wp-content/uploads/guides/CRSP_US_Treasury_Database_Guide_for_SAS_ASCII_EXCEL_R.pdf

https://www.crsp.org/products/documentation/treasno-properties

# WRDS API

In [1]:
import wrds
file_key = open("../../keys/wrds_username.txt","r")
USERNAME = file_key.read()
file_key.close()


In [2]:
db = wrds.Connection(wrds_username=USERNAME)
#db.create_pgpass_file()

Loading library list...
Done


In [3]:
import pandas as pd
import numpy as np
from datetime import date

***

# Database quote

In [4]:
DATE = '2025-04-30'
SQL_QUERY = f'select * from crsp_m_treasuries.tfz_dly where caldt=\'{DATE}\''
data_quotes = db.raw_sql(SQL_QUERY)

In [5]:
# drop any NaNs, cast to int, pull out unique values
ids = (data_quotes['kytreasno']
         .dropna()
         .astype(int)
         .unique()
         .tolist())

id_list = ','.join(str(i) for i in ids)
SQL_QUERY = (
    "select * "
    "from crsp_m_treasuries.tfz_iss "
    f"where kytreasno in ({id_list})"
)
data_iss = db.raw_sql(SQL_QUERY)

rawdata = data_quotes.merge(data_iss, on='kytreasno')

In [6]:
# drop any NaNs, cast to int, pull out unique values
ids = (data_quotes['kytreasno']
         .dropna()
         .astype(int)
         .unique()
         .tolist())

id_list = ','.join(str(i) for i in ids)
SQL_QUERY = (
    "select * "
    "from crsp_m_treasuries.tfz_iss "
    f"where kytreasno in ({id_list})"
)
data_iss = db.raw_sql(SQL_QUERY)

In [7]:
rawdata = data_quotes.merge(data_iss, on='kytreasno')

***

In [8]:
from cmds.bond_utils import find_tips_nominal_pairs
pairs = find_tips_nominal_pairs(rawdata)
pairs.sort_index()

,kytreasno nominal,kytreasno tips
maturity_date,,
2040-02-15,206112,206476
2041-02-15,206226,206477
2042-02-15,206341,206478
2043-02-15,206592,206593
2044-02-15,206711,206712
2045-02-15,206826,206827
2046-02-15,206950,206951
2047-02-15,207067,207068
2048-02-15,207187,207188


In [9]:
KEYNO_tips = tuple(pairs.iloc[0,:])

SQL_QUERY = f'select * from crsp_m_treasuries.tfz_dly where kytreasno in {KEYNO_tips}'
ts = db.raw_sql(SQL_QUERY)
ts['caldt'] = pd.to_datetime(ts['caldt'])

SQL_QUERY = f'select * from crsp_m_treasuries.tfz_iss where kytreasno in {KEYNO_tips}'
info = db.raw_sql(SQL_QUERY)
info.index = info['kytreasno'].astype(int).astype(str)

***

In [10]:
px = ts.pivot(
    index='caldt',
    columns='kytreasno',
    values='tdnomprc'
).dropna()

px.columns = px.columns.map(lambda x: str(int(x)))

In [11]:
# 1) extract your two IDs and maturities
ids = (
    info['kytreasno']
      .drop_duplicates()
      .astype(int)
      .astype(str)
      .tolist()
)
maturities = pd.to_datetime(
    info['tmatdt']        
        .tolist()
)

# 2) define your observation‐date index
obs_idx = pd.to_datetime(px.index)

# 3) build the Tx2 years‐to‐maturity frame
years_to_mat = pd.DataFrame(
    {
        id_: (m - obs_idx) / pd.Timedelta(days=365.25)
        for id_, m in zip(ids, maturities)
    },
    index=obs_idx
)

In [12]:
from cmds.bond_calcs import ytm

df_ytm = pd.DataFrame(index=px.index, columns=px.columns, dtype=float)
for col in px:
    cpn = info.loc[col,'tcouprt']/100
    for dt in px.index:
        p = px.loc[dt,col]
        T = years_to_mat.loc[dt,col]
        df_ytm.loc[dt,col] = ytm(p,T,cpn)

***

# Save to Excel

In [13]:
### raw data
outfile = f'../data/treasury_ts_crsp_{DATE}.xlsx'
with pd.ExcelWriter(outfile) as writer:
    info.T.to_excel(writer, sheet_name= 'info', index=True)
    years_to_mat.to_excel(writer, sheet_name= 'ttm', index=True)
    df_ytm.to_excel(writer, sheet_name= 'ytm', index=True)
    px.to_excel(writer, sheet_name= 'prices (clean)', index=True)

***